<a href="https://colab.research.google.com/github/EdmilsonSantana/llm-vehicle-repair/blob/main/Assistente_do_Mecanico_TCC.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [7]:
from docx import Document
from docx.text.paragraph import Paragraph
from docx.shared import Pt
import os
import re

### Processando apostilas da Universidade do Automóvel

In [54]:
RAW_DIR = './raw_data'
BASE_DIR = './data'

In [55]:
files = os.listdir(RAW_DIR)

In [56]:
print(f'{len(files)} documents')

14 documents


In [57]:
files

['5 -Motor de Comb. Interna - Otto.docx',
 '6 - Sistema de Transmissão.docx',
 '7 - Metrologia Mec. Automotiva.docx',
 '8 - Suspensão, Direção e Freios.docx',
 'Apostila Alternadores Pilotados V1 em 06_01_2023.docx',
 'Apostila de Injeção Eletrônica 2022.docx',
 'Apostila do Sistema de Iluminação e Sinalização V2 em 15_01_2024.docx',
 'Apostila Eletricidade Automotiva Diesel Pesados V.1 em 05_10_2023.docx',
 'Apostila Eletrônica Embarcada V2 em 16_08_2023.docx',
 'Apostila Introdução ao Euro 06 - Pesados.docx',
 'Apostila Módulo -  Eletricidade Aplicada.docx',
 'Apostila Pós-Tratamento Diesel Leve Euro 06 - Ducato.docx',
 'Apostila Sistema Ger. Motores Stellantis em 13_08_2023.docx',
 'Eletricidade - Sistema de Carga e Partida Versao 2024.1.docx']

In [58]:
document = Document(f'{RAW_DIR}/{files[0]}')

In [59]:
def has_small_font(paragraph: Paragraph):
    font_size = paragraph.style.font.size
    return font_size == Pt(9) or font_size == Pt(8)

In [60]:
def is_heading(paragraph: Paragraph):
    return paragraph.style.name.startswith('Heading')


In [61]:
def parse_text(text):
    text = re.sub('\s+',' ',text).strip()
    return '' if text.isnumeric() else text

In [62]:
doc_content = {}
curr_heading = "TITLE"
last_text = curr_heading

for section in document.sections:
    header_paragraphs = section.header.paragraphs
    footer_paragraphs = section.footer.paragraphs

    for item in section.iter_inner_content():
        if (not isinstance(item, Paragraph)):
            continue

        text = parse_text(item.text)
        if (len(text) == 0):
            continue
        if (item in header_paragraphs or item in footer_paragraphs):
            continue
        if (has_small_font(item)):
            continue

        if (is_heading(item)):
            curr_heading = text

        section_content = doc_content.setdefault(curr_heading, [])
        section_content.append(text)
            

In [63]:
doc_content.keys()

dict_keys(['TITLE', 'CICLO OTTO', 'Conteúdos', 'INTRODUÇÃO', 'MOTOR DE COMBUSTÃO', 'MOTOR DE COMBUSTÃO EXTERNA', 'MOTOR DE COMBUSTÃO INTERNA', 'MOTORES DE DOIS TEMPOS', 'MOTORES DE QUATRO TEMPOS', 'CABEÇOTE DO MOTOR', 'TIPOS DE CABEÇOTE', 'DISTRIBUIÇÃO MECÂNICA', 'DIAGRAMA DE VÁLVULAS', 'POSIÇÃO DO COMANDO E TIPO DE MOTOR', 'ESTRUTURA DA DISTRIBUIÇÃO MECÂNICA', 'ÁRVORE DE COMANDO DE VÁLVULAS', 'TUCHOS', 'CONJUNTO DE BALANCINS', 'MANUTENÇÃO', 'VÁLVULAS', 'CONSTITUIÇÃO DA VÁLVULA', 'DISPOSITIVOS DE MONTAGEM DAS VÁLVULAS', 'PEÇAS MÓVEIS', 'BIELA', 'CASQUILHO', 'ÊMBOLO', 'ANÉIS DE SEGMENTO', 'ÁRVORE DE MANIVELAS', 'BLOCO DE MOTOR', 'TIPOS DE BLOCO', 'VOLANTE DO MOTOR', 'FUNÇÕES DO VOLANTE DO MOTOR', 'DIMENSÕES E DESEMPENHO DO MOTOR', 'LUBRIFICAÇÃO', 'ATRITO', 'LUBRIFICANTES', 'SISTEMA DE LUBRIFICAÇÃO', 'SISTEMA DE LUBRIFICAÇÃO MISTO', 'SISTEMA DE CÁRTER SECO', 'CÁRTER', 'FILTRO DE ÓLEO', 'BOMBA DE ÓLEO', 'VÁLVULA REGULADORA DE PRESSÃO', 'BIBLIOGRAFIA'])

In [64]:
doc_content.pop('Conteúdos')
doc_content.pop('BIBLIOGRAFIA')

['BIBLIOGRAFIA',
 'https://www.mmcofap.com.br',
 'CASTROL. Veículos automotivos e sua lubrificação.',
 'COFAP. Manual doutor em motores. 2a.ed., 1988.',
 'FIAT. Oficina FIAT no. 2, vol. II, ano2. 1987. Tutela lubrificantes: história, informações, técnicas e Produtos.',
 'MOBIL. Curso de lubrificação. São Paulo, 1975.']

In [65]:
doc_content.keys()

dict_keys(['TITLE', 'CICLO OTTO', 'INTRODUÇÃO', 'MOTOR DE COMBUSTÃO', 'MOTOR DE COMBUSTÃO EXTERNA', 'MOTOR DE COMBUSTÃO INTERNA', 'MOTORES DE DOIS TEMPOS', 'MOTORES DE QUATRO TEMPOS', 'CABEÇOTE DO MOTOR', 'TIPOS DE CABEÇOTE', 'DISTRIBUIÇÃO MECÂNICA', 'DIAGRAMA DE VÁLVULAS', 'POSIÇÃO DO COMANDO E TIPO DE MOTOR', 'ESTRUTURA DA DISTRIBUIÇÃO MECÂNICA', 'ÁRVORE DE COMANDO DE VÁLVULAS', 'TUCHOS', 'CONJUNTO DE BALANCINS', 'MANUTENÇÃO', 'VÁLVULAS', 'CONSTITUIÇÃO DA VÁLVULA', 'DISPOSITIVOS DE MONTAGEM DAS VÁLVULAS', 'PEÇAS MÓVEIS', 'BIELA', 'CASQUILHO', 'ÊMBOLO', 'ANÉIS DE SEGMENTO', 'ÁRVORE DE MANIVELAS', 'BLOCO DE MOTOR', 'TIPOS DE BLOCO', 'VOLANTE DO MOTOR', 'FUNÇÕES DO VOLANTE DO MOTOR', 'DIMENSÕES E DESEMPENHO DO MOTOR', 'LUBRIFICAÇÃO', 'ATRITO', 'LUBRIFICANTES', 'SISTEMA DE LUBRIFICAÇÃO', 'SISTEMA DE LUBRIFICAÇÃO MISTO', 'SISTEMA DE CÁRTER SECO', 'CÁRTER', 'FILTRO DE ÓLEO', 'BOMBA DE ÓLEO', 'VÁLVULA REGULADORA DE PRESSÃO'])

In [66]:
doc_content['TITLE'].extend(doc_content['CICLO OTTO'])
doc_content.pop('CICLO OTTO')

['CICLO OTTO', '.']

### Extraindo instruções a partir do conteúdo

In [39]:
! modal deploy inference.py

- Creating objects...
| Creating objects...
/ Creating objects...
├── - Creating mount e:\Workspace\tcc-2022-2\inference.py: Uploaded 0/0 
│   inspected files
└── - Creating mount PythonPackage:prompt_template: Uploaded 0/0 inspected 
    files
\ Creating objects...
├── \ Creating mount e:\Workspace\tcc-2022-2\inference.py: Uploaded 0/0 
│   inspected files
└── \ Creating mount PythonPackage:prompt_template: Uploaded 0/0 inspected 
    files
/ Creating objects...
├── / Creating mount e:\Workspace\tcc-2022-2\inference.py: Uploaded 0/0 
│   inspected files
└── / Creating mount PythonPackage:prompt_template: Uploaded 0/0 inspected 
    files
\ Creating objects...
├── \ Creating mount e:\Workspace\tcc-2022-2\inference.py: Uploaded 0/0 
│   inspected files
└── \ Creating mount PythonPackage:prompt_template: Uploaded 0/0 inspected 
    files
/ Creating objects...
├── / Creating mount e:\Workspace\tcc-2022-2\inference.py: Uploaded 0/0 
│   inspected files
└── / Creating mount PythonPackage:pr

In [85]:
section = 'MOTOR DE COMBUSTÃO EXTERNA'
document = " ".join(doc_content[section])
document

'MOTOR DE COMBUSTÃO EXTERNA Na locomotiva a vapor, o combustível é o carvão ou a lenha. O calor produzido é utilizado para aquecer água em uma caldeira, transformando-a em vapor. O vapor se expande e, com sua pressão, vai movimentar os êmbolos que acionam as rodas motrizes da locomotiva. fornal A locomotiva a vapor é movida por um motor de combustão externa, pois a queima do combustível ocorre fora dos compartimentos que produzem o movimento (cilindros). Seguindo o princípio de funcionamento da locomotiva a vapor, foram desenvolvidos os primeiros triciclos a vapor, como o do francês Cugnot. Veja representado na figura a seguir.'

In [116]:
import modal
import json
from prompt_template import system_prompt

inference = modal.Function.lookup('llm-inference', 'Model.generate')

response = inference.remote(system_prompt, document)

In [117]:
dataset = json.loads(response)
dataset

[{'query': 'Quanto o combustível utilizado na locomotiva a vapor é?',
  'answer': 'O combustível é o carvão ou a lenha.'},
 {'query': 'Por que o calor é utilizado na locomotiva?',
  'answer': 'Para aquecer água em uma caldeira, transformando-a em vapor.'},
 {'query': 'Como o vapor move a locomotiva?',
  'answer': 'O vapor, com sua pressão, viaja pelos êmbolos que acionam as rodas motrizes.'},
 {'query': 'O motor de combustão externa na locomotiva a vapor está localizado onde?',
  'answer': 'Fora dos compartimentos que produzem o movimento, no cilindro.'},
 {'query': 'Quem foi responsável pelo desenvolvimento de primeiros triciclos a vapor?',
  'answer': 'O francês Cugnot.'},
 {'query': 'Representarão na figura a seguir o que?',
  'answer': 'Os primeiros triciclos a vapor.'}]

In [118]:
dataset_path = f"{BASE_DIR}/dataset.json"

In [119]:
if os.path.getsize(dataset_path) > 0:
    with open(dataset_path, "r", encoding='utf-8') as f:
        dataset.extend(json.load(f))

In [120]:
print(len(dataset))

17


In [121]:
with open(dataset_path, "w", encoding='utf-8') as f:
    json.dump(dataset, f, ensure_ascii=False)